## Pairing Based Cryptography

要implement此類crypto至少需要以下計算:

1. Arithmetic in $Z_p$: 可以建構在 GMP library 上 (提供了 number theoretic functions 像是 inversion modulo a prime 和  Jacobi symbol 等)

2. Elliptic curve groups: 解 $y^2=x^3+ax+b over Zp$, point addition 和 multiplication (可參考前一份notebook)

3. Bilinear pairing: Miller’s algorithm

上述可以實現出 Type A and B (and A1) pairings- 快速但是elements 需要很多空間來表示，如果要實現起他 pairing 就必須實現 Field extensions  

> 另外還要尋找 pairing-friendly curves。 像是 MNT curve construction method 需要 finding roots modulo a given prime, testing polynomial irreducibility, computing Hilbert polynomials. 這些都需要 high precision complex floating point arithmetic 和解 Pell-type equation

> 另外我們要隨機在 elliptic curve 上產生點，常見做法是隨機選擇 x-coordinate 然後解上述的 quadratic equation 來得到 y. (若無解， 則再換下一個x座標)。對於 odd characteristics， 如果我們可以找到 square roots of elements這可以一次達成。 在 finite field of prime order 中 square roots 可以藉由解 Tonelli-Shanks algorithm來得到。 至於其他field 則要參考其他新的文獻。(一個解法是去解 $x^2−a$ 用 Cantor-Zassenhaus method (Legendre’s method))。

> 對於特定情況有更簡潔的方式可以產生隨機點。 例如若 p 是質數且 $p = 2 \pmod 3 $ 則 x 的三次方根(Fq中) 為 $x^{(2p−1)/3}$. 故若方程式中 a=0，我們可以選擇先挑y然後解x。

## Introduction to Charm

### Group Abstractions

實做scheme的第一步是去選擇適當的 group。 Modern cryptographic algorithms 通常建構在提供特定數學難題性質的group上。

在charm中提供了三種cryptographic settings：
1. Integergroups
2. ecgroups
3. pairinggroups

要 initialize a group in the elliptic curve (EC) setting, 請參考 toolbox.eccurve 中實做的成員和方法以及了解提供相關的 NIST approved curves (e.g., prime192v1). 

對 EC with billinear maps (or pairings)來說，提供了 symmetric 和 asymmetric 的 curves。 例如: 'SS512' 是一個 symmetric curve with a 512-bit base field 而'MNT159' 代表了 asymmetric curve with 159-bit base field. 

> 注意這些 curves 是 **prime order**。

最後，對於 Integer groups, 通常定義大質數 p and q 就足以產生一個 RSA group. 若要用其他像 schnorr groups, 可能會需要一些時間來產生相關參數，因為 **他們要求safe primes (例如: p = 2q + 1)**. 接下來我們呈現 integer 和 pairing groups:

In [1]:
from charm.toolbox.integergroup import IntegerGroup

In [2]:
group1 = IntegerGroup()
group1.paramgen(1024)

g = group1.randomGen()
g, type(g)

(126718448021159160359934194216188095011790673518005716410613543976340376974278448597790040322847143185433823665947065637251234257564106575743327925315904954382029410181060274119654492467467774183754126146625693787497308897378581695879881453929327673070841693575677968169801425342568443195419423231977141754097 mod 146387814195932295811560398007442941509458707484784815804609238347035622430759630713176807594240242584931794104219240551716512180979843746398733268419038482881533449536896341597914992638585710806314664438710916425069737942346080470960996393474483523953020825613327803479363327437117739164965953523999292861663,
 integer.Element)

In [3]:
dir(g)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imod__',
 '__imul__',
 '__init__',
 '__int__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__itruediv__',
 '__le__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__pow__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__xor__',
 'isCoPrime',
 'isCongruent',
 'set']

In [4]:
from charm.toolbox.pairinggroup import PairingGroup,ZR,G1,G2,GT,pair

In [5]:
group2 = PairingGroup('SS512')
g1 = group2.random(G1)
g2 = group2.random(G2)

In [6]:
g1, g2, type(g1)

([2251021704668406136088146871006530026846893698475890645666363261293403196332004278537290526145717405038710071673290796822594018723099992097248009610699466, 8688876508258939519243795439777296838356618304489807431400070599193325115516183043074990852499908990948942843609788265350584904262664376503615708296580130],
 [6758123494869216936709646680834919013415475770376483777454167823268182453261911643122742240807730364186107488571827177842175136324980567083086904271378617, 6390593899781115686086873936571112849813387941676454537589652218906564478997157473777061152544271434973259767484961113431593428291541630778825739579273745],
 pairing.Element)

In [7]:
help(g1)

Help on Element object:

class Element(builtins.object)
 |  Pairing element objects
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __iadd__(self, value, /)
 |      Return self+=value.
 |  
 |  __imul__(self, value, /)
 |      Return self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __int__(self, /)
 |      int(self)
 |  
 |  __invert__(self, /)
 |      ~self
 |  
 |  __ipow__(self, value, /)
 |      Return self**=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __itruediv__(self, value, /)
 |      Return self/=value.
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__

In [8]:
type(g2)

pairing.Element

### Implement a Scheme

通常採用OOP的實作方法來增進可重複利用以及延伸性。 事實上內建了許多 base classes 對許多 cryptographic primitives 含有 standard interfaces 像是 PKEnc(public-key encryption)， PKSig(public-key signatures)， ABEnc(attribute-based encryption) 等等。**第一步是去繼承這些class** (下面我們說明如何用 Charm 實現 Cramer-Shoup PKEnc scheme。)

> 在 charm toolbox 中每種 cryptographic setting 有一個對應的 group abstraction 像是前面提到的 elliptic curve group , pairing group, 和 integer groups。 這些 abstractions 提供了 convenient 且 simple 的界面來選擇 group parameters, 運算 group operations, 和 benchmarking.

接下來，針對所要實現的 cryptographic scheme **要 import 我們要實現相關的 group setting**。

在 class initialization時，會呼叫 __init__， 在這個例子中會定義PKEnc基本的 security properties: 把 NIST standard elliptic curve identifier當成input。另外， 在這例子中，我們將 group object 定義為 global variable. 

> 另一種方法是定義 group 為class member，例如: self.group = ECGroup(curve).

In [9]:
from charm.toolbox.PKEnc import PKEnc
from charm.toolbox.ecgroup import ECGroup

class CS98(PKEnc):
    def __init__(self, curve):
        PKEnc.__init__(self)
        global group
        group = ECGroup(curve)

接下來看 Keygen， 他只接受一個 security parameter，然後產生 public 和 private keys 給 user， 注意這邊我們需要選一個 hash function H

$\begin{array}{c}
    g_1, g_2 \in G \\
    x_1, x_2, y_1, y_2, z \in Z_q \\
    c = g_1^{x_1} \cdot g_2^{x_2}, d = g_1^{y_1} \cdot g_2^{y_2}, h = g_1^z \\
    pk = (g_1, g_2, c, d, h, H) \\
    sk = (x_1, x_2, y_1, y_2, z)
\end{array}$

In [10]:
def keygen(self, secparam): # Security parameter only
    g1, g2 = group.random(G), group.random(G)
    x1, x2, y1, y2, z = group.random(ZR), group.random(ZR), group.random(ZR), group.random(ZR), group.random(ZR)
    c = (g1 ** x1) * (g2 ** x2)
    d = (g1 ** y1) * (g2 ** y2)
    h = (g1 ** z)

    pk = { 'g1' : g1, 'g2' : g2, 'c' : c, 'd' : d, 'h' : h, 'H' : group.hash } #dict class
    sk = { 'x1' : x1, 'x2' : x2, 'y1' : y1, 'y2' : y2, 'z' : z }
    return (pk, sk)

encryption 在 paper中敘述如下，我們利用group的encode幫我們轉換message到運算domain

$\begin{array}{c}
    m \in G, r \in Z_q \\
    u_1 = g_1^r, u_2 = g_2^r, e = h^r \cdot m , \alpha = H(u_1, u_2, e), v = c^r \cdot d^{r\alpha} \\
    (u_1, u_2, e, v) 
\end{array}$

In [11]:
def encrypt(self, pk, m):
    r   = group.random(ZR)
    u1  = pk['g1'] ** r
    u2  = pk['g2'] ** r
    e   = group.encode(m) * (pk['h'] ** r)
    alpha = pk['H'](u1, u2, e)
    v   = (pk['c'] ** r) * (pk['d'] ** (r * alpha))

    return { 'u1' : u1, 'u2' : u2, 'e' : e, 'v' : v }

decryption 在 paper中敘述如下

$\begin{array}{c}
    \alpha = H(u_1, u_2, e) \\
    v = u_1^{x_1+y_1\alpha} \cdot u_2^{x_2+y_2\alpha} \\
    m = e/{u_1^z} 
\end{array}$

In [12]:
def decrypt(self, pk, sk, c):
    alpha = pk['H'](c['u1'], c['u2'], c['e'])

    v_prime = (c['u1'] ** (sk['x1'] + (sk['y1'] * alpha))) * (c['u2'] ** (sk['x2'] + (sk['y2'] * alpha)))
    if (c['v'] != v_prime):
        return 'reject'
    return group.decode(c['e'] / (c['u1'] ** sk['z']))

這類scheme會將 messages 定義為 group element，故我們利用charm內建的 encode/decode methods 來將message轉換進入 G。 然而目前沒有支援直接轉進pairing group的 encode/decode。 因此將用別種方式來替代直接轉換

## Reuse Tools
先在這邊找找看有沒有可用的工具吧! https://jhuisi.github.io/charm/toolbox.html#toolbox

### Test and Benchmark
設計好shceme之後需要做驗證和量測performance。 charm提供了兩個 approaches: 

- Define a test routine that executes the algorithms in your scheme via test vectors if they exist a
- **embedding the test routine as a docstring in your scheme’s class definition**. 

> Docstrings 可用下列方法直接執行! ``python -m doctest myScheme.py``

另外有下列benchmark flag可以使用: **RealTime, CpuTime, Add, Sub, Mul, Div, and Exp**. 以下是對 EC setting 使用 Charm 的 benchmark interface

In [13]:
from charm.toolbox.ecgroup import ECGroup,ZR,G
from charm.toolbox.eccurve import prime192v1

trials = 10
group = ECGroup(prime192v1)
g = group.random(G)
h = group.random(G)
i = group.random(G)

assert group.InitBenchmark(), "failed to initialize benchmark"
group.StartBenchmark(["Mul", "Div", "Exp", "Granular"])
for a in range(trials):
    j = g * h
    k = h ** group.random(ZR)
    t = (j ** group.random(ZR)) / k
group.EndBenchmark()

msmtDict = group.GetGeneralBenchmarks()
print ("<=== General Benchmarks ===>")
print ("Mul := ", msmtDict["Mul"])
print ("Div := ", msmtDict["Div"])
print ("Exp := ", msmtDict["Exp"])
granDict = group.GetGranularBenchmarks()
print ("<=== Granular Benchmarks ===>")
print ("G mul   := ", granDict["Mul"][G])
print ("G exp   := ", granDict["Exp"][G])

<=== General Benchmarks ===>
Mul :=  10
Div :=  10
Exp :=  20
<=== Granular Benchmarks ===>
G mul   :=  10
G exp   :=  20


以上的 benchmark function 也可以用在其他的 group settings。 pairing base module同樣支援 **granular level (operation count per group)**的 benchmark。 如下:

In [14]:
from charm.toolbox.pairinggroup import PairingGroup,ZR,G1,G2,GT,pair

trials = 10
group = PairingGroup("SS1024")
g = group.random(G1)
h = group.random(G1)
i = group.random(G2)

assert group.InitBenchmark(), "failed to initialize benchmark"
group.StartBenchmark(["Mul", "Exp", "Pair", "Granular"])
for a in range(trials):
    j = g * h
    k = i ** group.random(ZR)
    t = (j ** group.random(ZR)) / h
    n = pair(h, i)
group.EndBenchmark()

msmtDict = group.GetGeneralBenchmarks()
granDict = group.GetGranularBenchmarks()
print("<=== General Benchmarks ===>")
print("Results  := ", msmtDict)
print("<=== Granular Benchmarks ===>")
print("G1 mul   := ", granDict["Mul"][G1])
print("G2 exp   := ", granDict["Exp"][G2])

<=== General Benchmarks ===>
Results  :=  {'Add': 0, 'Exp': 20, 'Div': 0, 'Pair': 10, 'Mul': 10, 'CpuTime': 0.0, 'Sub': 0, 'RealTime': 0.0}
<=== Granular Benchmarks ===>
G1 mul   :=  10
G2 exp   :=  10


另外針對 integer module， 我們可以 benchmarking without a group object:

In [15]:
from charm.core.math.integer import *
trials = 10
a = integer(1234)

assert InitBenchmark(), "failed to initialize benchmark"
StartBenchmark(["RealTime", "Exp", "Mul"])
for k in range(trials):
    r = randomPrime(512)
    s = r * (r ** a)
    j = r * (r ** a)
EndBenchmark()

msmtDict1 = GetGeneralBenchmarks()
print ("General Benchmarks: ", msmtDict1)

General Benchmarks:  {'Add': 0, 'Exp': 20, 'Div': 0, 'Mul': 20, 'CpuTime': 0.0, 'Sub': 0, 'RealTime': 0.153699}


## Optimization
針對 pairing base module， charm提供了 pre-computation tables 來加速 group exponentiation。 要使用的話，先呼叫 `initPP()` method 在 pairing object in G1, G2, or GT 上。 `initPP()` 會儲存 pre-computed values。 見以下比較10次 exponential 有用和沒有用的差別

In [16]:
from charm.toolbox.pairinggroup import PairingGroup,ZR,G1,G2,GT,pair

count = 10
group = PairingGroup("MNT224")
g = group.random(GT)
assert g.initPP(), "failed to init pre-computation table"
h = group.random(GT)
a, b = group.random(ZR, 2)

assert group.InitBenchmark(), "failed to initialize benchmark"
group.StartBenchmark(["RealTime"])
for i in range(count):
    A = g ** a
group.EndBenchmark()
print ("With PP: ", group.GetBenchmark("RealTime"))



assert group.InitBenchmark(), "failed to initialize benchmark"
group.StartBenchmark(["RealTime"])
for i in range(count):
    B = h ** b
group.EndBenchmark()
print ("Without: ", group.GetBenchmark("RealTime"))

With PP:  0.0039
Without:  0.022098


## Implement Application

拉高一個 abstraction level 來看，我們可以在應用中拿內建的sheme(https://jhuisi.github.io/charm/schemes.html#schemes)。 每一個內建scheme都有example的main function吃 default的test。 因此我們可以看main來了解如何使用它， 底下為在application中使用 Cramer-Shoup scheme:

In [17]:
from charm.schemes.pkenc.pkenc_cs98 import CS98
from charm.toolbox.eccurve import prime192v1
from charm.toolbox.ecgroup import ECGroup

groupObj = ECGroup(prime192v1)
pkenc = CS98(groupObj)

(pk, sk) = pkenc.keygen()

M = b'Hello World!'
ciphertext = pkenc.encrypt(pk, M)

message = pkenc.decrypt(pk, sk, ciphertext)
(ciphertext, message)

({'e': [491750628481418803602493522010939058512036971758417264302, 6027390645016250781082412440918421485655789926614433235604],
  'u1': [1857476881424227546302697882386080616110484101053772334153, 4143999534637894579530291758516509813789445391955291868696],
  'u2': [3567833863139880730546011706019261902031859474140046502360, 1144532107149268674365367065558755186081823873210741661695],
  'v': [4252544097880324260374018465787155639262423472841079645810, 58621827712002745818319214738851036403104134531967622465]},
 b'Hello World!')

### Serial API
為了支援 key或 ciphertext的 serialization，charm toolbox 中提供了兩個 high-level API 來 serialize charm objects (和 python structures (e.g., lists, tuples, or dictionaries, etc)做轉換) 亦即`charm.core.engine.util` packag 中的 `objectToBytes()` 和 `bytesToObject()`。 這些function可以將 keys 和 ciphertexts 轉換成 **base 64 encoded strings**。

以下說明如何使用 upported group objects (integergroup, pairinggroup or ecgroup)的這個 API:

```
from charm.core.engine.util import objectToBytes,bytesToObject

pk_bytes = objectToBytes(pk, group)
orig_pk = bytesToObject(pk_bytes, group)
```

若想要加入 custom serialization routine，以下說明 schemes based on the integergroup 如何在不需要 group object下達成:

```
from charm.core.math.integer import integer,serialize,deserialize

class mySerializeAPI:
        def __init__(self)
                ...

        def serialize(self, charm_object):
            assert type(charm_object) == integer, "required type is integer, not: ", type(charm_object)
            return serialize(charm_object)

        def deserialize(self, object):
            assert type(object) == bytes, "required type is bytes, not: ", type(object)
            return deserialize(object)

from charm.core.engine.util import objectToBytes,bytesToObject

serObject = mySerializeAPI()
pk_bytes = objectToBytes(pk, serObject)
orig_pk = bytesToObject(pk_bytes, serObject)
```

## ABE example

In [18]:
from charm.schemes.abenc.abenc_bsw07 import CPabe_BSW07
from charm.toolbox.pairinggroup import PairingGroup,GT

debug = True
groupObj = PairingGroup('SS512')

cpabe = CPabe_BSW07(groupObj)
attrs = ['THREE', 'ONE', 'TWO']
access_policy = '((ONE or THREE) and (ONE or FOUR))'
if debug:
    print("Attributes =>", attrs); print("Policy =>", access_policy)

(pk, mk) = cpabe.setup()

sk = cpabe.keygen(pk, mk, attrs)
print("sk :=>", sk)

rand_msg = groupObj.random(GT)
if debug: print("msg =>", rand_msg)
ct = cpabe.encrypt(pk, rand_msg, access_policy)

if debug: print("\n\nCiphertext...\n")
groupObj.debug(ct)

rec_msg = cpabe.decrypt(pk, sk, ct)
if debug: print("\n\nDecrypt...\n")
if debug: print("Rec msg =>", rec_msg)

assert rand_msg == rec_msg, "FAILED Decryption: message is incorrect"
if debug: print("Successful Decryption!!!")

Attributes => ['THREE', 'ONE', 'TWO']
Policy => ((ONE or THREE) and (ONE or FOUR))
sk :=> {'Djp': {'THREE': [2891643987289506771429640795284614363316891494402903331472663646478103766288600231975108722404270343234840928042535420077641217495927903589418868456270693, 8081657435447258940000010088378982647010384737505222468660280887221290559576241125075575470385855154326771011201276472278963536034122381178384913040649285], 'TWO': [388526237479738603992410148870954227856376946329079868818437984129831891152801859202339800495040561781259165065173023511653400446082823730963468073986136, 3248617884528091925911708623354329682436781316737426664728615659081290847388503047220015338836642883835569736417190095931908834449068268453829717413605037], 'ONE': [3148684062761606271735893752702914818801234575605744005533241818815974804456024775980838150064227071581088467960940961822920048405089005222546719343685656, 7615143648694482091724749198748715054598719352101248251388447555423228846328473160655276435731

In [19]:
from charm.toolbox.pairinggroup import PairingGroup,ZR,G1,G2,GT,pair
from charm.toolbox.secretutil import SecretUtil
from charm.toolbox.ABEnc import ABEnc

class CPabe09(ABEnc):
    """
    >>> from charm.toolbox.pairinggroup import PairingGroup,GT
    >>> group = PairingGroup('SS512')
    >>> cpabe = CPabe09(group)
    >>> msg = group.random(GT)
    >>> (master_secret_key, master_public_key) = cpabe.setup()
    >>> policy = '((ONE or THREE) and (TWO or FOUR))'
    >>> attr_list = ['THREE', 'ONE', 'TWO']
    >>> secret_key = cpabe.keygen(master_public_key, master_secret_key, attr_list)
    >>> cipher_text = cpabe.encrypt(master_public_key, msg, policy)
    >>> decrypted_msg = cpabe.decrypt(master_public_key, secret_key, cipher_text)
    >>> decrypted_msg == msg
    True
    """
    
    def __init__(self, groupObj):
        ABEnc.__init__(self)
        global util, group
        util = SecretUtil(groupObj, debug)        
        group = groupObj
                        
    def setup(self):
        g1, g2 = group.random(G1), group.random(G2)
        alpha, a = group.random(), group.random()        
        e_gg_alpha = pair(g1,g2) ** alpha
        msk = {'g1^alpha':g1 ** alpha, 'g2^alpha':g2 ** alpha}        
        pk = {'g1':g1, 'g2':g2, 'e(gg)^alpha':e_gg_alpha, 'g1^a':g1 ** a, 'g2^a':g2 ** a}
        return (msk, pk)
    
    def keygen(self, pk, msk, attributes):        
        t = group.random()
        K = msk['g2^alpha'] * (pk['g2^a'] ** t)
        L = pk['g2'] ** t
        k_x = [group.hash(s, G1) ** t for s in attributes]
        
        K_x = {}
        for i in range(0, len(k_x)):
            K_x[ attributes[i] ] = k_x[i]    

        key = { 'K':K, 'L':L, 'K_x':K_x, 'attributes':attributes }
        return key
    
    def encrypt(self, pk, M, policy_str):
        # Extract the attributes as a list
        policy = util.createPolicy(policy_str)        
        p_list = util.getAttributeList(policy)
        s = group.random()
        C_tilde = (pk['e(gg)^alpha'] ** s) * M
        C_0 = pk['g1'] ** s
        C, D = {}, {}
        secret = s
        shares = util.calculateSharesList(secret, policy)

        # ciphertext
        for i in range(len(p_list)):
            r = group.random()
            if shares[i][0] == p_list[i]:
               attr = shares[i][0].getAttribute() 
               C[ p_list[i] ] = ((pk['g1^a'] ** shares[i][1]) * (group.hash(attr, G1) ** -r))
               D[ p_list[i] ] = (pk['g2'] ** r)
        
        if debug: print("SessionKey: %s" % C_tilde)
        return { 'C0':C_0, 'C':C, 'D':D , 'C_tilde':C_tilde, 'policy':policy_str, 'attribute':p_list }
    
    def decrypt(self, pk, sk, ct):
        policy = util.createPolicy(ct['policy'])
        pruned = util.prune(policy, sk['attributes'])
        if pruned == False:
            return False
        coeffs = util.getCoefficients(policy)
        numerator = pair(ct['C0'], sk['K'])
        
        # create list for attributes in order...
        k_x, w_i = {}, {}
        for i in pruned:
            j = i.getAttributeAndIndex()
            k = i.getAttribute()
            k_x[ j ] = sk['K_x'][k]
            w_i[ j ] = coeffs[j]
            #print('Attribute %s: coeff=%s, k_x=%s' % (j, w_i[j], k_x[j]))
            
        C, D = ct['C'], ct['D']
        denominator = 1
        for i in pruned:
            j = i.getAttributeAndIndex()
            denominator *= ( pair(C[j] ** w_i[j], sk['L']) * pair(k_x[j] ** w_i[j], D[j]) )   
        return ct['C_tilde'] / (numerator / denominator)

In [20]:
from charm.toolbox.pairinggroup import PairingGroup,GT
debug = True
group = PairingGroup('SS512')
cpabe = CPabe09(group)
msg = group.random(GT)
(master_secret_key, master_public_key) = cpabe.setup()
policy = '((ONE or THREE) and (TWO or FOUR))'
attr_list = ['THREE', 'ONE', 'TWO']
secret_key = cpabe.keygen(master_public_key, master_secret_key, attr_list)
cipher_text = cpabe.encrypt(master_public_key, msg, policy)
decrypted_msg = cpabe.decrypt(master_public_key, secret_key, cipher_text)
decrypted_msg == msg

SessionKey: [374081390243472460414255142214019168642664288964032589151348346326788405201900105520548082275752271006101404827987566470502946194264005495495548745864962, 922485868452072429572737217329939172690950473107730927512300780908444090016036347053120673945732902115508937564737316230458880546100480991652102906278652]


True

## Appendix - CryptoBase

In [21]:
from charm.toolbox.paddingschemes import PKCS7Padding
from charm.toolbox.securerandom import OpenSSLRand
from charm.core.crypto.cryptobase import MODE_CBC,AES,selectPRP
import json
from base64 import b64encode,b64decode

class SymmetricCryptoAbstraction(object):
    """
    Abstraction for symmetric encryption and decryption of data.
    Ideally provide an INDCCA2 secure symmetric container for arbitrary data.
    Currently only supports primitives that JSON can encode and decode.
  
    A large number of the schemes can only encrypt group elements 
    and do not provide an efficient mechanism for encoding byte in
    those elements. As such we don't pick a symmetric key and encrypt 
    it asymmetrically. Rather, we hash a random group element to get the
    symmetric key.
    >>> from charm.toolbox.pairinggroup import PairingGroup,GT
    >>> groupObj = PairingGroup('SS512')
    >>> from charm.core.math.pairing import hashPair as extractor
    >>> a = SymmetricCryptoAbstraction(extractor(groupObj.random(GT)))
    >>> ct = a.encrypt(b"Friendly Fire Isn't")
    >>> a.decrypt(ct)
    b"Friendly Fire Isn't"
    """

    def __init__(self,key, alg = AES, mode = MODE_CBC):
        self._alg = alg
        self.key_len = 16
        self._block_size = 16 
        self._mode = mode
        self._key = key[0:self.key_len]
        self._padding = PKCS7Padding();
 
    def _initCipher(self,IV = None):
        if IV == None :
            IV =  OpenSSLRand().getRandomBytes(self._block_size)
        self._IV = IV
        return selectPRP(self._alg,(self._key,self._mode,self._IV))

    def __encode_decode(self,data,func):
        data['IV'] = func(data['IV'])
        data['CipherText'] = func(data['CipherText'])
        return data

    #This code should be factored out into  another class
    #Because json is only defined over strings, we need to base64 encode the encrypted data
    # and convert the base 64 byte array into a utf8 string
    def _encode(self,data):
        return self.__encode_decode(data,lambda x:b64encode(x).decode('utf-8'))

    def _decode(self,data):
        return self.__encode_decode(data,lambda x:b64decode(bytes(x,'utf-8')))

    def encrypt(self, message):
        #This should be removed when all crypto functions deal with bytes"
        if type(message) != bytes :
            message = bytes(message,"utf-8")
        ct = self._encrypt(message)
        #JSON strings cannot have binary data in them, so we must base64 encode  cipher
        cte = json.dumps(self._encode(ct))
        return cte

    def _encrypt(self,message):
        #Because the IV cannot be set after instantiation, decrypt and encrypt 
        # must operate on their own instances of the cipher 
        cipher = self._initCipher() 
        ct= {'ALG':self._alg,
            'MODE':self._mode,
            'IV':self._IV,
            'CipherText':cipher.encrypt(self._padding.encode(message))
            }
        return ct

    def decrypt(self,cipherText):
        f = json.loads(cipherText)
        return self._decrypt(self._decode(f)) #.decode("utf-8")

    def _decrypt(self,cipherText):
        cipher = self._initCipher(cipherText['IV'])
        msg = cipher.decrypt(cipherText['CipherText'])
        return self._padding.decode(msg)

In [8]:
from charm.toolbox.pairinggroup import PairingGroup,GT
groupObj = PairingGroup('SS512')
from charm.core.math.pairing import hashPair as extractor
a = SymmetricCryptoAbstraction(extractor(groupObj.random(GT)))
ct = a.encrypt(b"Friendly Fire Isn't")
a.decrypt(ct)

b"Friendly Fire Isn't"

In [22]:
%reload_ext version_information
%version_information numpy, scipy, matplotlib, charm, version_information

Software versions
Python 3.4.3+ 64bit [GCC 5.2.1 20151010]
IPython 4.0.0
OS Linux 4.2.0 30 generic x86_64 with Ubuntu 15.10 wily
numpy 1.10.4
scipy 0.14.1
matplotlib 1.4.2
charm The 'charm' distribution was not found and is required by the application
version_information The 'version_information' distribution was not found and is required by the application
Wed Jul 20 11:40:20 2016 UTC